**In this guide we will demonstrate how to use Nano AutoML to auto tune an machine learning model and handle the whole model development process easily.**

# Introduction
Nano provides hpo as an solution for fast and high-performance hyperparameter tuning. Users can import hano hpo following the standard tensorflow API without changing their own code. By setting the search space and running model.search(), the model will contain the best hyperparameters.

Nano hpo supports sequential, fuctional and custom keras API.


# Step0: Prepare Environment
We recommend using conda to prepare the environment.
conda create -n my_env python=3.7

conda activate my_env

pip install --pre --upgrade bigdl

# Step1: Init Nano AutoML
Let's start out by importing Nano AutoML and enable its tensorflow API globally by automl.hpo_config.enable_hpo_tf().

In [1]:
import bigdl.nano.automl as automl
import bigdl.nano.automl.hpo as hpo
automl.hpo_config.enable_hpo_tf()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import RMSprop

../../src/bigdl/nano/tf/__init__.py:24: UserWarning: NANO_TF_INTER_OP not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("NANO_TF_INTER_OP not found the in os.environ, "
../../src/bigdl/nano/tf/__init__.py:30: UserWarning: OMP_NUM_THREADS not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("OMP_NUM_THREADS not found the in os.environ, "


# Step2: Prepare data
We will load MNIST as our data.

In [2]:
CLASSES = 10

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
img_x, img_y = x_train.shape[1], x_train.shape[2]
x_train = x_train.reshape(-1, img_x, img_y,1).astype("float32") / 255
x_test = x_test.reshape(-1, img_x, img_y,1).astype("float32") / 255

# Step3: Set search space
We now create our custom model class. 

Whenever you want to search a hyperparameter, you can simply replace it with a space and set your own searching space 


In [3]:
@hpo.tfmodel()
class MyModel(tf.keras.Model):

    def __init__(self, filters, kernel_size, strides, activation):
        super().__init__()
        self.conv1 = Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            activation=activation)
        self.flat = Flatten()
        self.dense = Dense(CLASSES, activation="softmax")

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.flat(x)
        x = self.dense(x)
        return x
model = MyModel(
    filters=hpo.space.Categorical(32, 64),
    kernel_size=hpo.space.Categorical(3, 5),
    strides=hpo.space.Categorical(1, 2),
    activation=hpo.space.Categorical("relu", "linear")
)
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=["accuracy"],
)

2022-05-19 17:07:11.583495: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-19 17:07:11.584441: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


# Step4: Run hyperparameter search
Apply hyperparameter searching by calling model.search(), and best hyperparameters will be saved in the model.

You can specify the number of trial, each trial will use a set of hyperparameters automatiacally suggusted by nano.hpo. 

You can set your target metric and whehter you want to maximize you minimize it.

In [4]:
%%time
model.search(
    n_trials=10,
    target_metric='val_accuracy',
    direction="maximize",
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=24,
    validation_split=0.2,
    verbose=False,
)

[I 2022-05-19 17:07:11,598] A new study created in memory with name: no-name-5e6e44d1-edc3-42f6-bfa9-cc9139729d79


Starting a new tuning
Epoch 1/24
375/375 - 3s - loss: 0.9656 - accuracy: 0.8048 - val_loss: 0.3965 - val_accuracy: 0.8972 - 3s/epoch - 9ms/step
Epoch 2/24
375/375 - 3s - loss: 0.3699 - accuracy: 0.8945 - val_loss: 0.3106 - val_accuracy: 0.9107 - 3s/epoch - 8ms/step
Epoch 3/24
375/375 - 3s - loss: 0.3241 - accuracy: 0.9076 - val_loss: 0.2939 - val_accuracy: 0.9163 - 3s/epoch - 8ms/step
Epoch 4/24
375/375 - 3s - loss: 0.3069 - accuracy: 0.9124 - val_loss: 0.2858 - val_accuracy: 0.9205 - 3s/epoch - 7ms/step
Epoch 5/24
375/375 - 2s - loss: 0.2973 - accuracy: 0.9155 - val_loss: 0.2773 - val_accuracy: 0.9237 - 2s/epoch - 6ms/step
Epoch 6/24
375/375 - 3s - loss: 0.2905 - accuracy: 0.9173 - val_loss: 0.2765 - val_accuracy: 0.9237 - 3s/epoch - 9ms/step
Epoch 7/24
375/375 - 2s - loss: 0.2859 - accuracy: 0.9187 - val_loss: 0.2737 - val_accuracy: 0.9242 - 2s/epoch - 6ms/step
Epoch 8/24
375/375 - 5s - loss: 0.2821 - accuracy: 0.9199 - val_loss: 0.2691 - val_accuracy: 0.9258 - 5s/epoch - 14ms/step
E

[I 2022-05-19 17:08:18,102] Trial 0 finished with value: 0.9291666746139526 and parameters: {'activation▁choice': 1, 'filters▁choice': 1, 'kernel_size▁choice': 0, 'strides▁choice': 1}. Best is trial 0 with value: 0.9291666746139526.


Epoch 1/24
375/375 - 4s - loss: 0.9531 - accuracy: 0.8056 - val_loss: 0.3774 - val_accuracy: 0.8957 - 4s/epoch - 9ms/step
Epoch 2/24
375/375 - 5s - loss: 0.3592 - accuracy: 0.8980 - val_loss: 0.2975 - val_accuracy: 0.9161 - 5s/epoch - 12ms/step
Epoch 3/24
375/375 - 5s - loss: 0.3046 - accuracy: 0.9138 - val_loss: 0.2684 - val_accuracy: 0.9248 - 5s/epoch - 13ms/step
Epoch 4/24
375/375 - 4s - loss: 0.2718 - accuracy: 0.9233 - val_loss: 0.2429 - val_accuracy: 0.9330 - 4s/epoch - 11ms/step
Epoch 5/24
375/375 - 4s - loss: 0.2437 - accuracy: 0.9313 - val_loss: 0.2204 - val_accuracy: 0.9392 - 4s/epoch - 10ms/step
Epoch 6/24
375/375 - 3s - loss: 0.2188 - accuracy: 0.9385 - val_loss: 0.1995 - val_accuracy: 0.9457 - 3s/epoch - 9ms/step
Epoch 7/24
375/375 - 3s - loss: 0.1965 - accuracy: 0.9445 - val_loss: 0.1795 - val_accuracy: 0.9512 - 3s/epoch - 9ms/step
Epoch 8/24
375/375 - 3s - loss: 0.1767 - accuracy: 0.9500 - val_loss: 0.1640 - val_accuracy: 0.9568 - 3s/epoch - 8ms/step
Epoch 9/24
375/375 -

[I 2022-05-19 17:09:50,683] Trial 1 finished with value: 0.9780833125114441 and parameters: {'activation▁choice': 0, 'filters▁choice': 1, 'kernel_size▁choice': 1, 'strides▁choice': 1}. Best is trial 1 with value: 0.9780833125114441.


...

Epoch 1/24
375/375 - 11s - loss: 0.5452 - accuracy: 0.8696 - val_loss: 0.3040 - val_accuracy: 0.9110 - 11s/epoch - 28ms/step
Epoch 2/24
375/375 - 10s - loss: 0.3085 - accuracy: 0.9115 - val_loss: 0.2832 - val_accuracy: 0.9198 - 10s/epoch - 27ms/step
Epoch 3/24
375/375 - 11s - loss: 0.2907 - accuracy: 0.9180 - val_loss: 0.2770 - val_accuracy: 0.9221 - 11s/epoch - 29ms/step
Epoch 4/24
375/375 - 11s - loss: 0.2828 - accuracy: 0.9195 - val_loss: 0.2720 - val_accuracy: 0.9247 - 11s/epoch - 29ms/step
Epoch 5/24
375/375 - 9s - loss: 0.2777 - accuracy: 0.9218 - val_loss: 0.2727 - val_accuracy: 0.9256 - 9s/epoch - 24ms/step
Epoch 6/24
375/375 - 12s - loss: 0.2745 - accuracy: 0.9229 - val_loss: 0.2648 - val_accuracy: 0.9263 - 12s/epoch - 32ms/step
Epoch 7/24
375/375 - 12s - loss: 0.2711 - accuracy: 0.9244 - val_loss: 0.2656 - val_accuracy: 0.9259 - 12s/epoch - 33ms/step
Epoch 8/24
375/375 - 9s - loss: 0.2692 - accuracy: 0.9249 - val_loss: 0.2620 - val_accuracy: 0.9283 - 9s/epoch - 24ms/step
Epoc

[I 2022-05-19 17:26:49,000] Trial 8 finished with value: 0.9300833344459534 and parameters: {'activation▁choice': 1, 'filters▁choice': 1, 'kernel_size▁choice': 0, 'strides▁choice': 0}. Best is trial 6 with value: 0.9822499752044678.


Epoch 1/24
375/375 - 8s - loss: 0.7002 - accuracy: 0.8482 - val_loss: 0.3010 - val_accuracy: 0.9147 - 8s/epoch - 20ms/step
Epoch 2/24
375/375 - 8s - loss: 0.2880 - accuracy: 0.9180 - val_loss: 0.2409 - val_accuracy: 0.9329 - 8s/epoch - 21ms/step
Epoch 3/24
375/375 - 9s - loss: 0.2340 - accuracy: 0.9335 - val_loss: 0.2006 - val_accuracy: 0.9444 - 9s/epoch - 23ms/step
Epoch 4/24
375/375 - 14s - loss: 0.1939 - accuracy: 0.9454 - val_loss: 0.1694 - val_accuracy: 0.9544 - 14s/epoch - 37ms/step
Epoch 5/24
375/375 - 9s - loss: 0.1624 - accuracy: 0.9546 - val_loss: 0.1458 - val_accuracy: 0.9620 - 9s/epoch - 25ms/step
Epoch 6/24
375/375 - 9s - loss: 0.1381 - accuracy: 0.9620 - val_loss: 0.1287 - val_accuracy: 0.9655 - 9s/epoch - 23ms/step
Epoch 7/24
375/375 - 13s - loss: 0.1197 - accuracy: 0.9678 - val_loss: 0.1151 - val_accuracy: 0.9698 - 13s/epoch - 34ms/step
Epoch 8/24
375/375 - 8s - loss: 0.1060 - accuracy: 0.9711 - val_loss: 0.1046 - val_accuracy: 0.9713 - 8s/epoch - 21ms/step
Epoch 9/24
3

[I 2022-05-19 17:30:36,655] Trial 9 finished with value: 0.9821666479110718 and parameters: {'activation▁choice': 0, 'filters▁choice': 0, 'kernel_size▁choice': 1, 'strides▁choice': 0}. Best is trial 6 with value: 0.9822499752044678.


CPU times: user 1h 44min 50s, sys: 1h 19min 14s, total: 3h 4min 4s
Wall time: 23min 25s


# Step 5: fit with the best hyperparameters
The best hyperparameters have been saved in the model. You can call model.fit() to train a model with the best hyperparameters.

In [5]:
history = model.fit(x_train, y_train,
                    batch_size=128, epochs=24, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/24
375/375 [==============================] - 9s 24ms/step - loss: 0.6703 - accuracy: 0.8503 - val_loss: 0.3092 - val_accuracy: 0.9122
Epoch 2/24
375/375 [==============================] - 9s 23ms/step - loss: 0.2982 - accuracy: 0.9145 - val_loss: 0.2549 - val_accuracy: 0.9269
Epoch 3/24
375/375 [==============================] - 9s 25ms/step - loss: 0.2475 - accuracy: 0.9298 - val_loss: 0.2190 - val_accuracy: 0.9398
Epoch 4/24
375/375 [==============================] - 11s 28ms/step - loss: 0.2090 - accuracy: 0.9406 - val_loss: 0.1841 - val_accuracy: 0.9504
Epoch 5/24
375/375 [==============================] - 9s 25ms/step - loss: 0.1763 - accuracy: 0.9503 - val_loss: 0.1578 - val_accuracy: 0.9572
Epoch 6/24
375/375 [==============================] - 11s 29ms/step - loss: 0.1502 - accuracy: 0.9580 - val_loss: 0.1384 - val_accuracy: 0.9635
Epoch 7/24
375/375 [==============================] - 10s 26ms/step - loss: 0.1296 - accuracy: 0.9639 - val_loss: 0.1209 - val_accuracy: 0.9

Check out the summary of the model. The model has already been built with the best hyperparameters found by nano hpo.

In [6]:
print(model.summary())

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           multiple                  832       
                                                                 
 flatten_1 (Flatten)         multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  184330    
                                                                 
Total params: 185,162
Trainable params: 185,162
Non-trainable params: 0
_________________________________________________________________
None
